In [1]:
from pybullet_environment import *
import numpy as np
import cvxpy as cp

pybullet build time: Jul 17 2023 13:38:55


In [20]:
env = RobotEnvironment()

argv[0]=
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=3
argv[0] = --unused
argv[1] = 
argv[2] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 525.116.04
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 525.116.04
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


## Start Joint Configuration:

#### From End-effector pose

In [21]:
start = np.array([0.5,  0.3,  0.2])
start_orientation = np.array([3.14, 0, 1.57])

joint_config = env.inverse_kinematics(start, start_orientation)
print(joint_config)

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

(0.5295944018839492, 1.7078429357776748, -0.030811500746328113, 0.5722936008432751, 0.032876890574846454, 1.1134821847667495, -0.23722034050664934, 0.0, 0.0)


: 

#### From Joint angles

In [3]:
joint_config = np.array([-1.9300085,  1.3141115,  2.1577177, -1.5229155, -0.7427585, 1.4192199, -1.3984755])

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

## End Joint Configuration:

#### From End-effector pose

In [5]:
goal = np.array([0.5,  0.01,  0.4])
goal_orientation = np.array([3.14, 0, 0])

joint_config = env.inverse_kinematics(goal, goal_orientation)
print(joint_config)

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

(0.2916356471941378, -0.19476593195471914, -0.2515576632025949, -2.0901947227739055, -0.05264943377223378, 1.9009807816043083, 0.8471365555475125, 0.0, 0.0)


#### From Joint angles

In [5]:
joint_config = np.zeros((9,))

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

# -------------------------------------------------------------------------------

In [15]:
lower_limit = np.array([-2.8973000049591064, -1.7627999782562256, -2.8973000049591064, -3.0717999935150146, -2.8973000049591064, -0.017500000074505806, -2.8973000049591064])

joint_config = np.zeros((9,))
n = 1
joint_config[n] = lower_limit[n]
#joint_config = np.array([0.7, -0.2, 0.3, -1.8, 0., 2.1, 1.57, 0., 0.])

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

In [16]:
upper_limit = np.array([2.8973000049591064, 1.7627999782562256, 2.8973000049591064, -0.0697999969124794, 2.8973000049591064, 3.752500057220459, 2.8973000049591064])
joint_config = np.zeros((9,))
n = 1
joint_config[n] = upper_limit[n]
#joint_config = np.array([0.7, -0.2, 0.3, -1.8, 0., 2.1, 1.57, 0., 0.])

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

In [4]:
joint_config = np.array([-0.7856447,  0.7866443,  2.689896 , -1.9357901, -0.3345264,
        1.2200228, -2.0265737])
for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

In [5]:
a = p.calculateJacobian(env.manipulator, 1, [0, 0, 0], np.zeros((9,)).tolist(), np.zeros((9,)).tolist(), np.zeros((9,)).tolist())
np.array(a).shape
print(a)

(((0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)), ((0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), (1.0, 4.896638650109253e-12, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)))


In [5]:
p.disconnect()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


In [4]:
print(env.get_joint_velocities())

[0. 0. 0. 0. 0. 0. 0.]


In [11]:
_, _ = env.move_joints(np.zeros((7,)))

In [26]:
_, _ = env.move_joints([1, 0.5, 0.7, -0.5, 1, 1, 1])

: 

In [ ]:
joint_config = np.array([-1.9300085,  1.3141115,  2.1577177, -1.5229155, -0.7427585, 1.4192199, -1.3984755])

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

In [ ]:
env.client_id.getLinkState(env.manipulator, 11, computeForwardKinematics = 1)

((0.41517849174779053, -0.37502881028017254, 0.5576215683161793),
 (0.45227439950028625,
  0.8300439147763229,
  0.3097416480331348,
  0.1026405308499977),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, 0.0, 1.0),
 (0.41517847776412964, -0.3750288188457489, 0.5576215386390686),
 (0.4522744119167328,
  0.8300439715385437,
  0.30974164605140686,
  0.1026405468583107))

In [ ]:
env.client_id.getBasePositionAndOrientation(env.manipulator)

((0.0, 0.0, 0.05), (0.0, 0.0, 0.0, 1.0))

In [ ]:
current_joints = [*env.get_joint_positions(), 0, 0]
print(current_joints)

[-1.9300085, 1.3141115, 2.1577177, -1.5229155, -0.7427585, 1.4192199, -1.3984755, 0, 0]


In [ ]:
bullet_fk = env.get_end_effector_transformation()
print(bullet_fk)

[[-0.56982558  0.68723113  0.45056873  0.41517849]
 [ 0.81439932  0.39901596  0.42135497 -0.37502881]
 [ 0.10978414  0.60704171 -0.78705007  0.55762157]
 [ 0.          0.          0.          1.        ]]


In [ ]:
fk = env.get_ee_tf_mat(current_joints)
print(fk)

[[-0.56982558  0.68723113  0.45056873  0.41445758]
 [ 0.81439932  0.39901596  0.42135497 -0.37570298]
 [ 0.10978414  0.60704171 -0.78705007  0.55888085]
 [ 0.          0.          0.          1.        ]]


numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished


In [ ]:
link_index = 11
com = env.client_id.getLinkState(env.manipulator, link_index, computeForwardKinematics=1)[2]
print(list(com))
theta = cp.Variable(9)

J_t, J_r = env.client_id.calculateJacobian(env.manipulator, 
                                link_index, 
                                list(com), 
                                current_joints, 
                                np.zeros_like(current_joints).tolist(), 
                                np.zeros_like(current_joints).tolist())

bullet_J = np.concatenate([np.array(J_t), np.array(J_r)], axis = 0)

[0.0, 0.0, 0.0]


In [ ]:
J = env.get_jacobian(current_joints)
#print(J - bullet_J[:, :-2])
#print(J, "\n\n", bullet_J[:, :-2])